In [ ]:
import pandas as pd
import os
import numpy as np
import plotly.express as px
import itertools
import matplotlib.pyplot as plt
import math
import plotly.graph_objects as go
import plotly.colors
from plotly.subplots import make_subplots        
#from PIL import ImageColor
import pickle
import time
from scipy.spatial import distance
from scipy.optimize import curve_fit
import dcor

from sklearn.pipeline import Pipeline, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.neighbors import LocalOutlierFactor


# for power law fit "model"
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense

In [ ]:
pd.set_option('display.max_rows', 500)

# Framework for automated evaluation of column tuples

The aim is to provide a routine that gets fed a dataframe / csv-file, runs data analysis routines and returns metrics and plots à la

`metrics, plot = predictability(df, num_input_columns, num_target_columns, list_of_considered_columns, "analysis method")`

The metrics we use* are 

| r2  |  RMSE | RMSE/std | MAPE  | RAE  | Distance correlation  |
| :-: | :-: | :-: | :-: | :-: | :-: |
| $$1-\frac{\sum (\hat{y}-y)^2}{\sum (\bar{y}-y)^2}$$ | $$\sqrt{\frac{1}{N} \sum (\hat{y}-y)^2 }$$ | $$\sqrt{\frac{1}{N} \sum \frac{(\hat{y}-y)^2}{\sigma_y{}^2} }$$ | $$\frac{1}{N}\sum \frac{\lvert \hat{y}-y \rvert}{\lvert y\rvert}$$ | $$\frac{\sum \lvert \hat{y}-y\rvert}{\sum \lvert \bar{y}-y\rvert }$$ |  [cf. documentation](https://dcor.readthedocs.io/en/latest/theory.html)<br/> [cf. paper](https://projecteuclid.org/journals/annals-of-statistics/volume-35/issue-6/Measuring-and-testing-dependence-by-correlation-of-distances/10.1214/009053607000000505.full)<br/> [cf. wiki](https://en.wikipedia.org/wiki/Distance_correlation) |

with

$y$: observed data<br/>
$\hat{y}$: predicted counterpart of $y$<br/>
$N$: number of $y$ and $\hat{y}$<br/>
$\bar{y}$: average of $y$<br/>
$\sigma_y$: error in measurement of $y$

*) Note that RMSE/std is not yet implemented as we don't work with data that includes measurement errors so far.

# Open questions

* Should primary keys be considered or do we assume the data(frame) is handed over properly also with respect to fixing possible primary keys?\
In the example of Country Indicators data: Year and Country is not a normal data column. But it also depends on the choice of what we want to investigate whether we drop them or fix one of them.
* Should input be possible as dataframe only, or also as csv- / txt-file?
* We should also go through all relevant permutations of the data tuples.\
If we have, e.g., four data columns [A, B, C, D] and want to analyse 2-2 connections, it does not suffice to only consider input=[A, B], output=[C, D]. There may well be no causal connection between any of A,B and any of C,D, but instead between C and D. So we need to consider all $\frac{N!}{I!\cdot O!}$ many combinations, given $N$ data columns, $I$-many inputs and $O$-many outputs.
* Currently, RMSE/std relies on the standard deviation of the test values $y$. One could of course also use the overall available target values – the combined train and test values.
* Decision on whether outliers are extracted or not is currently based on *test* score.
* Currently, also the metrics of the linear regression and power law fits are computed for the test data set only.
* Fitting in log-log-space results in asymmetric error distributions as predicitions higher than the actual value are squeezed and lower values spreaded.\
Use packages for powerlaw-fitting? "Normal" fitting of exponential laws is quite sensitive to small deviations.\
Also different treatment of errors $\epsilon$, $\eta$:

|  | log-log space  | | linear space | error in log-log space | error in linear space |
| :-: | :-: | :-: | :-: | :-: | :-: |
| linear in linear space | | | $$ y = C\cdot x^a + \epsilon$$  | | additive |
| linear in log-log space | $$ \log y = \log C + a\cdot\log x {\color{red}\oplus} \eta$$ | $\Leftrightarrow$ | $$ y = C\cdot x^a {\color{blue}\odot} \mathrm{e}^\eta $$ | $${\color{red}{\mathrm{additive}}}$$ | $${\color{blue}{\mathrm{multiplicative}}}$$ |

* So far, scaling is included as optional argument and is either done on all columns or none (depending on yes/no/try as the argument input where the latter means scaling is part of the GridSearchCV). However, it should be done column-wise and automated. So check range of input columns' values and then apply scaler automatically, assuming it never worsens the result.

### load example data

In [ ]:
df = pd.read_csv("processed_country_indicators.csv").drop(columns=["Unnamed: 0"])

In [ ]:
df

In [ ]:
# to work with 2007 data only
df2007 = df.loc[df["Year"]==2007]
df2007

In [ ]:
#### primary keys

In [ ]:
The above dataset contains the two primary keys "Country Name" and "Year" which are not supposed to be part of the analysis.

Primary keys can be used to "zoom in" on more detailed analyses. In this case, e.g., check how numbers for Afghanistan evolved over time. Or only take data from year 2007 and find a connection between two columns, then compare it to the same data but for 2002 etc.

Therefore, the primary keys have to be treated differently from the remaining *data columns*.

In [ ]:
prim_keys = ["Country Name", "Year"]


# Helper routines

### relative absolute error

In [ ]:

def rae(true, predicted):
    numerator = np.sum(np.abs(predicted - true))
    denominator = np.sum(np.abs(np.mean(true) - true))
    return numerator / denominator


### 3d plotting

In [ ]:
def plot_3d_result(data_tuple, metrics, datas, show=False):
    
    
    '''fig = make_subplots(
    rows=3, cols=1,
    row_heights=[0.7, 0.15, 0.15],
    specs=[[{"type": "scatter3d"}],
           [{"type": "scatter", "t": -.07}],
           [{"type": "histogram", "t": .05}]],
    vertical_spacing=.1,
    #shared_xaxes = True
    )'''
    
    fig = go.Figure(data=[
        go.Scatter3d(x=[i[0] for i in datas[data_tuple]["X_train"]],
                     y=[i[1] for i in datas[data_tuple]["X_train"]],
                     z=datas[data_tuple]["y_train"].flatten(),
                     #xaxis="x",
                     #yaxis="y",
                     name="training data",
                     mode="markers",
                     marker_color=datas[data_tuple]["y_train"].flatten(),
                     marker_size=3,
                     opacity=.6
                    ),
        #row=1, col=1
                         ]
                   )
    
     
    fig.update_layout(
        title=data_tuple[2]+'  vs.  '+data_tuple[0]+' & '+data_tuple[1],
        scene=dict(xaxis=dict(title=data_tuple[0],
            gridcolor='white',
            gridwidth=2,
            #type='log',
        ),
        yaxis=dict(
            title=data_tuple[1],
            gridcolor='white',
            gridwidth=2,
            #type='log',
        ),
        zaxis=dict(
            title=data_tuple[2],
            gridcolor='white',
            gridwidth=2,
            #type='log',
        )
        ),
        #yaxis_range=[y_min*1.01,y_max*1.01],
        #xaxis2=dict(title=data_tuple[0], matches="x"),
        #yaxis2=dict(title="pred. error"),
        #xaxis3=dict(title=r"$\text{error } y_{pred}-y$"),
        #yaxis3=dict(title="frequency"),
        legend=dict(bgcolor="white"),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        width=920,
        height=720
    )
    
    if show==True:
        fig.show()
    else:
        return fig

### 2d plotting

In [ ]:
def plot_2d_result(data_tuple, metrics, datas, show=False):
    
    # need to order test values for correct connection of data points during line plotting
    pred_MLP_data = list(zip(datas[data_tuple]["X_test"].reshape(len(datas[data_tuple]["X_test"]),), 
             datas[data_tuple]["y_test_pred"]))
    pred_MLP_df = pd.DataFrame(data=pred_MLP_data,
             columns=["X_test", "y_test_pred"],
             ).sort_values(by="X_test")
    
    # get max and min plotting values
    y_min = min(0, min(datas[data_tuple]["y_train"]), min(datas[data_tuple]["y_test"]), min(datas[data_tuple]["y_test_pred"]))
    y_max = max(0, max(datas[data_tuple]["y_train"]), max(datas[data_tuple]["y_test"]), max(datas[data_tuple]["y_test_pred"]))
    
    fig = make_subplots(
    rows=3, cols=1,
    row_heights=[0.7, 0.15, 0.15],
    specs=[[{"type": "scatter"}],
           [{"type": "scatter", "t": -.07}],
           [{"type": "histogram", "t": .05}]],
    vertical_spacing=.1,
    #shared_xaxes = True
    )

    fig.add_trace(
        go.Scatter(x=datas[data_tuple]["X_train"].reshape(len(datas[data_tuple]["X_train"]),), 
                   y=datas[data_tuple]["y_train"].flatten(),
                   xaxis="x",
                   yaxis="y",
                   name="training data",
                   mode="markers", marker_color="Maroon", marker_size=3, opacity=.6
                    ),
        row=1, col=1
                 )
    fig.add_trace(
        go.Scatter(x=datas[data_tuple]["X_test"].reshape(len(datas[data_tuple]["X_test"]),), 
                   y=datas[data_tuple]["y_test"].flatten(),  
                   xaxis="x",
                   yaxis="y",
                   name="test data",
                   mode="markers", marker_color="LightSeaGreen", opacity=.8 
                    ),
        row=1, col=1
                 )
    fig.add_trace(
        go.Scatter(x=pred_MLP_df["X_test"],#.reshape(len(datas[data_tuple]["X_test"]),), 
                   y=pred_MLP_df["y_test_pred"], 
                   xaxis="x",
                   yaxis="y",
                   name="predictions MLP",
                   mode='lines+markers',
                   #mode="markers",
                   marker_color="Tomato", 
                    ),
        row=1, col=1
                 )
    # add power law predictions
    fig.add_trace(
        go.Scatter(x=datas[data_tuple]["X_test"].reshape(len(datas[data_tuple]["X_test"]),), 
                   y=datas[data_tuple]["y_test_pred_pl"].flatten(), 
                   xaxis="x",
                   yaxis="y",
                   name="predictions pl",
                   mode="markers", marker_color="SteelBlue", 
                    ),
        row=1, col=1
                 )
        
    # add metrics
    
    fig.add_annotation(text='mean y_train: '+str(round(datas[data_tuple]["y_test_pred_mean"][0])),
                      align="right",
                      showarrow=False,
                        xref='paper',
                        yref='paper',
                        x=1.223,
                        y=.73,
                        bgcolor="white")
    
    fig.add_annotation(text='<b>r2 MLP:   </b>'+str(round(metrics[data_tuple]["MLP r2"],2))+
                       ' <i><br>r2 pow. law:   </i>'+str(round(metrics[data_tuple]["pow. law r2"],2))+
                       ' <i><br>r2 lin. reg.:   </i>'+str(round(metrics[data_tuple]["linear r2"],2))+
                       ' <i><br>r2 mean pred.:   </i>'+str(round(metrics[data_tuple]["mean r2"],2))+
                       ' <b><br>RMSE MLP:   </b>'+str(round(metrics[data_tuple]["MLP RMSE"],2))+
                       ' <i><br>RMSE pow. law:   </i>'+str(round(metrics[data_tuple]["pow. law RMSE"],2))+
                       ' <i><br>RMSE lin. reg.:   </i>'+str(round(metrics[data_tuple]["linear RMSE"],2))+
                       ' <i><br>RMSE nean pred.:   </i>'+str(round(metrics[data_tuple]["mean RMSE"],2))+
                       ' <b><br>RMSE/std MLP:   </b>'+str(round(metrics[data_tuple]["MLP RMSE/std"],2))+
                       ' <i><br>RMSE/std pow. law:   </i>'+str(round(metrics[data_tuple]["pow. law RMSE/std"],2))+
                       ' <i><br>RMSE/std lin. reg.:   </i>'+str(round(metrics[data_tuple]["linear RMSE/std"],2))+
                       ' <i><br>RMSE/std mean pred.:   </i>'+str(round(metrics[data_tuple]["mean RMSE/std"],2))+
                       ' <b><br>MAPE MLP:   </b>'+str(round(metrics[data_tuple]["MLP MAPE"],2))+
                       ' <i><br>MAPE pow. law:   </i>'+str(round(metrics[data_tuple]["pow. law MAPE"],2))+
                       ' <i><br>MAPE lin. reg.:   </i>'+str(round(metrics[data_tuple]["linear MAPE"],2))+
                       ' <i><br>MAPE mean pred.:   </i>'+str(round(metrics[data_tuple]["mean MAPE"],2))+
                       ' <b><br>rae MLP:   </b>'+str(round(metrics[data_tuple]["MLP rae"],2))+
                       ' <i><br>rae pow. law:   </i>'+str(round(metrics[data_tuple]["pow. law rae"],2))+
                       ' <i><br>rae lin. reg.:   </i>'+str(round(metrics[data_tuple]["linear rae"],2))+
                       ' <i><br>rae mean pred.:   </i>'+str(round(metrics[data_tuple]["mean rae"],2))+
                       ' <b><br>dcor MLP:   </b>'+str(round(metrics[data_tuple]["MLP dcor"],2))+
                       ' <i><br>dcor pow. law:   </i>'+str(round(metrics[data_tuple]["pow. law dcor"],2))+
                       ' <i><br>dcor lin. reg.:   </i>'+str(round(metrics[data_tuple]["linear dcor"],2))+
                       ' <i><br>dcor mean pred.:   </i>'+str(round(metrics[data_tuple]["mean dcor"],2)),
                       #' <br>Spearman corr.:   '+str(round(metrics[data_tuple]["Spearman"],2))+
                       #' <br>Pearson corr.:   '+str(round(metrics[data_tuple]["Pearson"],2)), 
                        align='right',
                        showarrow=False,
                        xref='paper',
                        yref='paper',
                        x=1.223,
                        y=.68,
                        bgcolor="white",
                        #bordercolor='black',
                        #borderwidth=1
                      )

    # local plot of errors
    fig.add_trace(
        go.Scatter(x=datas[data_tuple]["X_test"].reshape(len(datas[data_tuple]["X_test"]),), 
                   y=datas[data_tuple]["y_test_pred"].flatten()-datas[data_tuple]["y_test"].flatten(), 
                   xaxis="x2",
                   yaxis="y2",
                   mode="markers", marker_color="Tomato",
                   legendgroup="MLP",
                   name="pred. error MLP" 
                    ),
        row=2, col=1
                 )
    fig.add_trace(
        go.Scatter(x=datas[data_tuple]["X_test"].reshape(len(datas[data_tuple]["X_test"]),), 
                   y=datas[data_tuple]["y_test_pred_pl"].flatten()-datas[data_tuple]["y_test"].flatten(), 
                   xaxis="x2",
                   yaxis="y2",
                   mode="markers", marker_color="SteelBlue",
                   legendgroup="pl",
                   name="pred. error pl" 
                    ),
        row=2, col=1
                 )
    
    # add line as separator
    fig.add_shape(type='line',
                x0=-.05,
                y0=.1,
                x1=1.05,
                y1=.1,
                line=dict(color='white',),
                xref='paper',
                yref='paper'
    )
    
    # histogram of errors
    fig.add_trace(
        go.Histogram(x=datas[data_tuple]["y_test_pred"]-datas[data_tuple]["y_test"].flatten(),
                   xaxis="x3",
                   yaxis="y3",
                   nbinsx=100,
                   marker_color='Tomato',
                   legendgroup="MLP",
                   name="pred. error MLP",
                    showlegend=False),
        row=3, col=1
                 )
    fig.add_trace(
        go.Histogram(x=datas[data_tuple]["y_test_pred_pl"].flatten()-datas[data_tuple]["y_test"].flatten(),
                   xaxis="x3",
                   yaxis="y3",
                   nbinsx=100,
                   marker_color='SteelBlue',
                   legendgroup="pl",
                   name="pred. error pl",
                    showlegend=False),
        row=3, col=1
                 )
    
    fig.update_layout(
        title=data_tuple[1]+'  vs.  '+data_tuple[0],
        xaxis=dict(
            gridcolor='white',
            gridwidth=2,
            #type='log',
        ),
        yaxis=dict(
            title=data_tuple[1],
            gridcolor='white',
            gridwidth=2,
            #type='log',
        ),
        yaxis_range=[y_min*1.01,y_max*1.01],
        xaxis2=dict(title=data_tuple[0],
                    matches="x"),
        yaxis2=dict(title="pred. error"),
        xaxis3=dict(title=r"$\text{error } y_{pred}-y$"),
        yaxis3=dict(title="frequency"),
        legend=dict(bgcolor="white"),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        width=920,
        height=720
    )
    fig.update_layout(legend_tracegroupgap=0)
    
    if show==True:
        fig.show()
    else:
        return fig

### outlier extraction

#### via class, if potentially inside pipeline

not in use!

In [ ]:
'''
class OutlierExtractor(TransformerMixin):
    def __init__(self, **kwargs):
        """
        Create a transformer to remove outliers. A threshold is set for selection
        criteria, and further arguments are passed to the LocalOutlierFactor class

        Keyword Args:
            neg_conf_val (float): The threshold for excluding samples with a lower
               negative outlier factor.

        Returns:
            object: to be used as a transformer method as part of Pipeline()
        """

        self.threshold = kwargs.pop('neg_conf_val', -10.0)

        self.kwargs = kwargs

    def transform(self, X, y):
        """
        Uses LocalOutlierFactor class to subselect data based on some threshold

        Returns:
            ndarray: subsampled data

        Notes:
            X should be of shape (n_samples, n_features)
        """
        X = np.asarray(X)
        y = np.asarray(y)
        lcf = LocalOutlierFactor(**self.kwargs)
        lcf.fit(X)
        return (X[lcf.negative_outlier_factor_ > self.threshold, :],
                y[lcf.negative_outlier_factor_ > self.threshold])

    def fit(self, *args, **kwargs):
        return self
'''

#### the standard way

In [ ]:
def extract_outliers(data):
    
    extractor = LocalOutlierFactor(n_neighbors=20)
    
    data_extr_pred = extractor.fit_predict(data)
    
    outliers_index = np.where(data_extr_pred==-1)
    outliers = data.iloc[outliers_index]
    inliers_index = np.where(data_extr_pred==1)
    data_extr = data.iloc[inliers_index]
    
    data_scores = extractor.negative_outlier_factor_
    
    return data_extr, data_scores, outliers

### data preparation, train-test-split

In [ ]:
def data_prep_split(data, inputs, outputs):
    # get x and y value(s)
    curr_x = np.array(data[inputs])#.reshape(-1, 1)
    curr_y = np.array(data[outputs])
    
    
    # train test split
    curr_X_train, curr_X_test, curr_y_train, curr_y_test = train_test_split(curr_x, curr_y, random_state=1,
                                                                            test_size=.3, shuffle=True)
    
    
    #curr_y_train = curr_y_train.ravel()
    #curr_y_test = curr_y_test.ravel()
    
    return curr_X_train, curr_X_test, curr_y_train, curr_y_test

### scoring mapping

In [ ]:
scoring_dict = {
    "r2": "r2",
    "MAPE": "neg_mean_absolute_percentage_error",
    "neg_mean_absolute_percentage_error": "neg_mean_absolute_percentage_error",
    "RMSE": "neg_root_mean_squared_error",
    "neg_root_mean_squared_error": "neg_root_mean_squared_error",
    "MAE": "neg_mean_absolute_error",
    "neg_mean_absolute_error": "neg_mean_absolute_error"
}

### get column combinations of desired number of input and output columns

In [ ]:
def get_column_combinations(all_cols, inputs, outputs):
    
    assert inputs+outputs <= len(all_cols), "More input and output columns specified than there are columns."
    
    # initialise final list of column combinations
    col_combinations = []
    # first, draw possible input tuples
    input_combinations = list(itertools.combinations(all_cols, inputs))
    # now go through all possible input combinations
    for i in input_combinations:
        # get list of possible output columns, i.e. columns that are not yet part of current input columns
        curr_output_cols = [o for o in all_cols if o not in i]
        # now draw from that list all currently possible output combinations
        output_combinations = list(itertools.combinations(curr_output_cols, outputs))
        # add all currently possible output combinations to the current input columns and save in final list
        for oc in output_combinations:
            col_combinations.append(i+(*oc,))
        
    return col_combinations

### power law fit

#### w/o errors

##### old

In [ ]:
def log_power_law_1_1(logx, logC, a1):
    return logC+a1*logx

In [ ]:
def power_law_1_1(x, logC, a1):
    return np.e**logC*x**a1

In [ ]:
def fit_predict_power_law_1_1(curr_X_train,
                      curr_X_test, 
                      curr_y_train, 
                      curr_y_test
                     ):
    
    # move data so everything is positive for fitting
    min_x = min(min(curr_X_train), min(curr_X_test))
    min_y = min(min(curr_y_train), min(curr_y_test))
    if min_x < 0:
        x_subtraction = 1.05*min_x
    elif min_x == 0:
        x_subtraction = -0.05
    else:
        x_subtraction = 0
    if min_y < 0:
        y_subtraction = 1.05*min_y
    elif min_y == 0:
        y_subtraction = -0.05
    else:
        y_subtraction = 0
    curr_X_train, curr_X_test = curr_X_train.flatten()-x_subtraction, curr_X_test.flatten()-x_subtraction
    curr_y_train, curr_y_test = curr_y_train.flatten()-y_subtraction, curr_y_test.flatten()-y_subtraction
    
    # values in log-space
    log_curr_X_train, log_curr_y_train = np.log(curr_X_train), np.log(curr_y_train)
    
    # fit
    fit_params, cov = curve_fit(log_power_law_1_1, log_curr_X_train, log_curr_y_train)
    
    curr_y_test_pred_pl = power_law_1_1(curr_X_test, *fit_params)+y_subtraction
    
    return curr_y_test_pred_pl

# The main routine

In [ ]:
def predictability(data, input_cols=1, output_cols=1, col_set=None, primkey_cols=[], targets=[],
                   method="MLP", hidden_layers=None, alphas=None, scoring="r2", scaling=True, 
                   max_iter=10000, n_jobs=-1, verbose=1):
    
    # TODO: map scoring to possible options
    scoring_dict = {
        "r2": "r2",
        "MAPE": "neg_mean_absolute_percentage_error",
        "neg_mean_absolute_percentage_error": "neg_mean_absolute_percentage_error",
        "RMSE": "neg_root_mean_squared_error",
        "neg_root_mean_squared_error": "neg_root_mean_squared_error",
        "MAE": "neg_mean_absolute_error",
        "neg_mean_absolute_error": "neg_mean_absolute_error"
    }
    scoring = scoring_dict[scoring]
    
    # if we want to measure the overall time
    start = time.time()
    
    # initialise the dictionary that is going to save the metrics per tuple
    metric_dict = {}
    
    # dict to save x-/y-train/-test and predicted values for subsequent plotting
    data_dict = {}
    
    # if primary keys are fed in, data columns should not contain these
    data_cols = [col for col in data.columns.to_list() if col not in primkey_cols]
    
    # if set of columns that should be considered is fed in, use this
    if col_set is not None:
        data_cols = list(set(col_set))
    
    # get the list of tuples of input and output columns
    if targets:
        data_tuples = get_column_combinations_w_targets(data_cols, input_cols, output_cols, targets)
    else:
        data_tuples = get_column_combinations(data_cols, input_cols, output_cols)    
    
    # for printing the progress of the analysis
    counter_tuples = 0
    
    # go through all tuples
    # or testing subset only:
    #data_tuples = [("Electric power consumption (kWh per capita)", "Life expectancy at birth, total (years)")]+data_tuples[:5]
    #
    for curr_tuple in data_tuples:
        
        # if we want to measure the current tuple's analysis time
        curr_start = time.time()
        
        print("Analysing "+str(curr_tuple)+" now.")
        
        # TODO: implement going through all permutations
        
        # get current inputs and outputs
        curr_inputs = list(curr_tuple[:input_cols])
        curr_outputs = list(curr_tuple[input_cols:])
        
        # reduce data to current columns and drop NAs
        curr_data = data[curr_inputs+curr_outputs].dropna()
        
        # dict of min values of current columns, used for power law fitting
        mi_dict = {}
        for col in curr_inputs+curr_outputs:
            curr_min = min(curr_data[col])
            if curr_min < 0:
                x_subtraction = 1.05*curr_min
            elif curr_min == 0:
                x_subtraction = -0.05
            else:
                x_subtraction = 0
            mi_dict[col] = x_subtraction
            
        
        # do data preparations and train-test-split
        curr_X_train, curr_X_test, curr_y_train, curr_y_test = data_prep_split(curr_data, curr_inputs, curr_outputs)
        
        # compute standard deviation of curr_y_test for later scaling of the RMSE
        curr_y_test_std = np.std(curr_y_test)
        
        #
        # y-mean "prediction"
        #
        curr_y_train_mean = np.mean(curr_y_train)
        curr_y_test_pred_mean = curr_y_train_mean*np.ones(len(curr_X_test))
        # metrics
        curr_mean_r2 = r2_score(curr_y_test, curr_y_test_pred_mean)
        curr_mean_rmse = mean_squared_error(curr_y_test, curr_y_test_pred_mean, squared=False)
        curr_mean_mape = mean_absolute_percentage_error(curr_y_test, curr_y_test_pred_mean)
        curr_mean_rae = rae(curr_y_test, curr_y_test_pred_mean)
        curr_mean_dcor = dcor.distance_correlation(curr_y_test, curr_y_test_pred_mean)
        
        #
        # linear regression
        #
        lin_reg = LinearRegression().fit(curr_X_train,curr_y_train)
        curr_y_test_pred = lin_reg.predict(curr_X_test)
        # metrics
        curr_lin_r2 = r2_score(curr_y_test, curr_y_test_pred)
        curr_lin_rmse = mean_squared_error(curr_y_test, curr_y_test_pred, squared=False)
        curr_lin_mape = mean_absolute_percentage_error(curr_y_test, curr_y_test_pred)
        curr_lin_rae = rae(curr_y_test, curr_y_test_pred)
        curr_lin_dcor = dcor.distance_correlation(curr_y_test, curr_y_test_pred)
        
        '''
        # power law fit so far only for 1-1 connections implemented
        if input_cols==1 and output_cols==1:
            #
            # power law fit
            #
            curr_y_test_pred_pl = fit_predict_power_law_1_1(curr_X_train, curr_X_test, curr_y_train, curr_y_test)

            # metrics
            curr_pl_r2 = r2_score(curr_y_test, curr_y_test_pred_pl)
            curr_pl_rmse = mean_squared_error(curr_y_test, curr_y_test_pred_pl, squared=False)
            curr_pl_mape = mean_absolute_percentage_error(curr_y_test, curr_y_test_pred_pl)
            curr_pl_rae = rae(curr_y_test, curr_y_test_pred_pl)
            curr_pl_dcor = dcor.distance_correlation(curr_y_test, curr_y_test_pred_pl)
            '''
        #
        # power law fit
        #
        if ((curr_X_train>0).all().all()) and ((curr_X_test>0).all().all()):
            do_pl_fit = True
        else:
            do_pl_fit = False
        
        if do_pl_fit:
            
            curr_X_train_log = curr_X_train.apply(lambda x: np.log(x))
            curr_X_test_log = curr_X_test.apply(lambda x: np.log(x))
            curr_y_train_log = curr_y_train.apply(lambda x: np.log(x))
            #curr_y_test_log = curr_y_test.apply(lambda x: np.log(x))
            
            pl_fit = LinearRegression().fit(curr_X_train_log, curr_y_train_log)
            
            curr_y_test_pred_pl = pl_fit.predict(curr_X_test_log)
            
            curr_y_test_pred_pl = np.exp(curr_y_test_pred_pl)
            
            # metrics
            curr_pl_r2 = r2_score(curr_y_test, curr_y_test_pred_pl)
            curr_pl_rmse = mean_squared_error(curr_y_test, curr_y_test_pred_pl, squared=False)
            curr_pl_mape = mean_absolute_percentage_error(curr_y_test, curr_y_test_pred_pl)
            curr_pl_rae = rae(curr_y_test, curr_y_test_pred_pl)
            curr_pl_dcor = dcor.distance_correlation(curr_y_test, curr_y_test_pred_pl)
            
        
        #
        # MLP regression
        print("start MLP routine")
        #
        # list of hidden layer sizes for GridSearch
        if hidden_layers is None:
            hidden_layers = [(12,), 
                              (50,), 
                              (70,5,), 
                              #(40,18,3,)
                            ]
        # list of alpha values for GridSearch
        if alphas is None:
            alphas = [0.001, 
                      0.0001, 
                      0.00001
                     ]

        # via pipeline (with and without scaler)
        if scaling=="yes":
            pipe = Pipeline([
                            ('scaler', StandardScaler()),
                            ('mlp', MLPRegressor(max_iter=max_iter))
                            ])
            pipe_params = [
                           {'mlp__hidden_layer_sizes': hidden_layers,
                            'mlp__alpha': alphas}
                            ]
            clf = GridSearchCV(pipe,
                               param_grid=pipe_params,
                               cv=3,
                               scoring=scoring,
                               return_train_score=True,
                               verbose=verbose,
                               n_jobs=n_jobs
                              )        
        elif scaling=="no":
            pipe = Pipeline([
                            ('scaler', StandardScaler()),
                            ('mlp', MLPRegressor(max_iter=max_iter))
                            ])
            pipe_params = [{'scaler': ['passthrough'],
                            'mlp__hidden_layer_sizes': hidden_layers,
                            'mlp__alpha': alphas}]
            clf = GridSearchCV(pipe,
                               param_grid=pipe_params,
                               cv=3,
                               scoring=scoring,
                               return_train_score=True,
                               verbose=verbose,
                               n_jobs=n_jobs
                              )
        else:
            pipe = Pipeline([
                            ('scaler', StandardScaler()),
                            ('mlp', MLPRegressor(max_iter=max_iter))
                            ])
            pipe_params = [{'scaler': ['passthrough'],
                            'mlp__hidden_layer_sizes': hidden_layers,
                            'mlp__alpha': alphas}, 
                           {'mlp__hidden_layer_sizes': hidden_layers,
                            'mlp__alpha': alphas}]
            clf = GridSearchCV(pipe,
                               param_grid=pipe_params,
                               cv=3,
                               scoring=scoring,
                               return_train_score=True,
                               verbose=verbose,
                               n_jobs=n_jobs
                              )
        
        clf.fit(curr_X_train, curr_y_train)
        
        curr_best_params = clf.best_params_
        curr_y_test_pred = clf.predict(curr_X_test)
        
        # metrics
        curr_mlp_r2 = r2_score(curr_y_test, curr_y_test_pred)
        curr_mlp_rmse = mean_squared_error(curr_y_test, curr_y_test_pred, squared=False)
        curr_mlp_mape = mean_absolute_percentage_error(curr_y_test, curr_y_test_pred)
        curr_mlp_rae = rae(curr_y_test, curr_y_test_pred)
        curr_mlp_dcor = dcor.distance_correlation(curr_y_test, curr_y_test_pred)

        # save metrics into dict
        # power law fit so far only for 1-1 connections implemented
        if do_pl_fit:
            metric_dict[curr_tuple] = {"MLP r2": curr_mlp_r2, "linear r2": curr_lin_r2, 
                                       "pow. law r2": curr_pl_r2, "mean r2": curr_mean_r2, 
                                        "MLP RMSE": curr_mlp_rmse, "linear RMSE": curr_lin_rmse,
                                        "pow. law RMSE": curr_pl_rmse, "mean RMSE": curr_mean_rmse,
                                        "MLP RMSE/std": curr_mlp_rmse/curr_y_test_std, "linear RMSE/std": curr_lin_rmse/curr_y_test_std,
                                       "pow. law RMSE/std": curr_pl_rmse/curr_y_test_std, "mean RMSE/std": curr_mean_rmse/curr_y_test_std,
                                        "MLP MAPE": curr_mlp_mape, "linear MAPE": curr_lin_mape,
                                       "pow. law MAPE": curr_pl_mape, "mean MAPE": curr_mean_mape,
                                        "MLP rae": curr_mlp_rae, "linear rae": curr_lin_rae,
                                        "pow. law rae": curr_pl_rae, "mean rae": curr_mean_rae,
                                        "MLP dcor": curr_mlp_dcor, "linear dcor": curr_lin_dcor,
                                        "pow. law dcor": curr_pl_dcor, "mean dcor": curr_mean_dcor,
                                               }
        else:
            metric_dict[curr_tuple] = {"MLP r2": curr_mlp_r2, "linear r2": curr_lin_r2, "mean r2": curr_mean_r2, 
                                        "MLP RMSE": curr_mlp_rmse, "linear RMSE": curr_lin_rmse, "mean RMSE": curr_mean_rmse,
                                        "MLP RMSE/std": curr_mlp_rmse/curr_y_test_std, "linear RMSE/std": curr_lin_rmse/curr_y_test_std,
                                        "mean RMSE/std": curr_mean_rmse/curr_y_test_std,
                                        "MLP MAPE": curr_mlp_mape, "linear MAPE": curr_lin_mape, "mean MAPE": curr_mean_mape,
                                        "MLP rae": curr_mlp_rae, "linear rae": curr_lin_rae, "mean rae": curr_mean_rae,
                                        "MLP dcor": curr_mlp_dcor, "linear dcor": curr_lin_dcor, "mean dcor": curr_mean_dcor,
                                               }

        # save values into dict
        # power law fit so far only for 1-1 connections implemented
        if do_pl_fit:
            
            data_dict[curr_tuple] = {"X_train": curr_X_train, "X_test": curr_X_test,
                                     "y_train": curr_y_train, "y_test": curr_y_test, "y_test_pred": curr_y_test_pred,
                                     "y_test_pred_pl": curr_y_test_pred_pl, "y_test_pred_mean": curr_y_test_pred_mean,
                                     "GridSearchParams": curr_best_params, "scores": clf.cv_results_
                                    }
        else:
            
            data_dict[curr_tuple] = {"X_train": curr_X_train, "X_test": curr_X_test,
                                     "y_train": curr_y_train, "y_test": curr_y_test, "y_test_pred": curr_y_test_pred,
                                     "y_test_pred_mean": curr_y_test_pred_mean,
                                     "GridSearchParams": curr_best_params, "scores": clf.cv_results_
                                    }
        
                
        # for printing the CV results per tuple
        #print(clf.cv_results_)
        
        print("The analysis of this tuple took "+str(round(time.time()-curr_start,2))+"s.")
        # for printing the progress of the analysis
        counter_tuples += 1
        print("-----"+str(counter_tuples)+"/"+str(len(data_tuples))+"-----")
    
    print("The whole run took "+str(round(time.time()-start,2))+"s.")
    
    return metric_dict, data_dict

In [ ]:
metrics, datas = predictability(data=df2007,
                                input_cols=1,
                                output_cols=1,
                                col_set = ['Electric power consumption (kWh per capita)', 
                                           'Agriculture, value added (% of GDP)',
                                           'Life expectancy at birth, total (years)', 
                                           'CO2 emissions (metric tons per capita)'],
                                primkey_cols = prim_keys,
                                scoring="RMSE"
                               )

In [ ]:
metrics_df = pd.DataFrame.from_dict(metrics).transpose()

# or 3d ones
#metrics_df = pd.DataFrame.from_dict(metrics3d).transpose()

metrics_df

In [ ]:
# 3-1 connections metrics
metrics3_1_df = pd.DataFrame.from_dict(metrics3_1).transpose()
# may show MLP metrics only
metrics3_1_df[["MLP r2", "MLP RMSE", "MLP MAPE", "MLP rae", "MLP dcor"]].sort_values(by="MLP r2", ascending=False)

### Run the plotting routine alone

issues:
* [-2]: r2 of power law fit = 0 ! But dcor is good. MAPE high due to one y-value = 0.
* [5]: r2 of MLP = 0 ! But actually, w.r.t. training data a rather good fit. dcor is good, too!
* [-3]: bad MLP fit, but MAPE rather good due to y-values far away from 0.
* [-4]: similar to above, MAPE again okay as high deviations exist mainly for predictions of high y-values

In [ ]:
plot_2d_result(list(datas.keys())[-2], metrics, datas, show=True)

In [ ]:
#plot_3d_result(list(datas.keys())[2], metrics, datas, show=True)
plot_3d_result(list(datas3d.keys())[11], metrics3d, datas3d, show=True)

### save dicts

In [ ]:
# 3d data
with open('metrics3d.pkl', 'wb') as f:
    pickle.dump(metrics, f)
with open('datas3d.pkl', 'wb') as f:
    pickle.dump(datas, f)
#with open('plots3d.pkl', 'wb') as f:
#    pickle.dump(plots, f)

In [ ]:
# 2d data
with open('metrics.pkl', 'wb') as f:
    pickle.dump(metrics, f)
with open('datas.pkl', 'wb') as f:
    pickle.dump(datas, f)
#with open('plots.pkl', 'wb') as f:
#    pickle.dump(plots, f)

### load dicts

In [ ]:
# 2d data
with open('metrics.pkl', 'rb') as f:
    metrics = pickle.load(f)
with open('datas.pkl', 'rb') as f:
    datas = pickle.load(f)
#with open('plots.pkl', 'rb') as f:
#    plots = pickle.load(f)


In [ ]:
# 3d data
with open('metrics3d.pkl', 'rb') as f:
    metrics3d = pickle.load(f)
with open('datas3d.pkl', 'rb') as f:
    datas3d = pickle.load(f)
#with open('plots3d.pkl', 'rb') as f:
#    plots = pickle.load(f)


In [ ]:
# data
with open('metrics3_1.pkl', 'rb') as f:
    metrics3_1 = pickle.load(f)
with open('datas3_1.pkl', 'rb') as f:
    datas3_1 = pickle.load(f)
#with open('plots3d.pkl', 'rb') as f:
#    plots = pickle.load(f)


#### or backups

In [ ]:


with open('metrics_backup.pkl', 'rb') as f:
    metrics_backup = pickle.load(f)
with open('datas_backup.pkl', 'rb') as f:
    datas_backup = pickle.load(f)
with open('plots_backup.pkl', 'rb') as f:
    plots_backup = pickle.load(f)


### Big run on 4-1 connections

In [ ]:
metrics_4_1, datas_4_1 = predictability(data=df2007,
                                input_cols=4,
                                output_cols=1,
                                primkey_cols = prim_keys,
                                scoring="RMSE"
                               )

In [ ]:
datas_4_1[('Agriculture, value added (% of GDP)',
  'CO2 emissions (metric tons per capita)',
  'Domestic credit provided by financial sector (% of GDP)',
  'Electric power consumption (kWh per capita)',
  'Exports of goods and services (% of GDP)')]

In [ ]:
with open('metrics_4_1.pkl', 'wb') as f:
    pickle.dump(metrics_4_1, f)
with open('datas_4_1.pkl', 'wb') as f:
    pickle.dump(datas_4_1, f)

## Big run to check dependency on scoring choice during GridSearchCV

In [ ]:
# big run through all 4 scorings
metrics_scorings = {}
for scoring in ["RMSE", "MAPE", "r2", "MAE"]:
    curr_metrics, curr_datas = predictability(data=df2007,
                                input_cols=1,
                                output_cols=1,
                                primkey_cols = prim_keys,
                                scoring=scoring
                               )
    metrics_scorings[scoring] = curr_metrics

In [ ]:
with open('metrics_scorings.pkl', 'wb') as f:
    pickle.dump(metrics_scorings, f)

In [ ]:
metrics_scorings_df = pd.DataFrame.from_dict(metrics_scorings)#.transpose()
RMSE_metrics = metrics_scorings_df['RMSE'].apply(pd.Series)[["MLP r2", "MLP RMSE", "MLP MAPE", "MLP rae", "MLP dcor"]]
RMSE_metrics.columns = pd.MultiIndex.from_product([["RMSE scoring"], RMSE_metrics.columns])
all_metrics_df = RMSE_metrics.copy(deep=True)
for scoring in ["r2", "MAPE", "MAE"]:
    curr_metrics_df = metrics_scorings_df[scoring].apply(pd.Series)[["MLP r2", "MLP RMSE", "MLP MAPE", "MLP rae", "MLP dcor"]]
    curr_metrics_df.columns = pd.MultiIndex.from_product([[scoring+" scoring"], curr_metrics_df.columns])
    all_metrics_df = pd.concat([all_metrics_df, curr_metrics_df], axis=1)
all_metrics_df

## Tests and checks

### all combinatinos for, e.g., 2-1 connections

In [ ]:
all_cols = ["a", "b", "c", 
            "d",# "e", "f"
           ]
inputs = 2
outputs = 1
def get_column_combinations(all_cols, inputs, outputs):
    
    assert inputs+outputs <= len(all_cols), "More input and output columns specified than there are columns."
    
    # initialise final list of column combinations
    col_combinations = []
    # first, draw possible input tuples
    input_combinations = list(itertools.combinations(all_cols, inputs))
    # now go through all possible input combinations
    for i in input_combinations:
        # get list of possible output columns, i.e. columns that are not yet part of current input columns
        curr_output_cols = [o for o in all_cols if o not in i]
        # now draw from that list all currently possible output combinations
        output_combinations = list(itertools.combinations(curr_output_cols, outputs))
        # add all currently possible output combinations to the current input columns and save in final list
        for oc in output_combinations:
            col_combinations.append(i+(*oc,))
        
    return col_combinations

In [ ]:
get_column_combinations(all_cols, 2, 1)

### other tests

In [ ]:
test_df = df2007[["Electric power consumption (kWh per capita)", "Life expectancy at birth, total (years)"]].dropna()
test_x = test_df[["Electric power consumption (kWh per capita)"]].values.flatten()
test_y = test_df[["Life expectancy at birth, total (years)"]].values.flatten()


In [ ]:
cols = ['Agriculture, value added (% of GDP)', 'Domestic credit provided by financial sector (% of GDP)']
test_df = df2007[cols].dropna()
x_subtraction = 1.05*min(test_df[cols[0]].values)-0.01
y_subtraction = 1.05*min(test_df[cols[1]].values)-0.01
test_x = test_df[cols[0]].values.flatten()-x_subtraction
test_y = test_df[cols[1]].values.flatten()-y_subtraction
print(x_subtraction, y_subtraction)

In [ ]:
test_params = fit_power_law_1_1(test_x, test_y)

In [ ]:
test_params

In [ ]:
pred_test_y = power_law_1_1(test_x, *test_params)

In [ ]:
plt.plot(test_x+x_subtraction, test_y+y_subtraction, "bo", label="data")
plt.plot(test_x+x_subtraction, pred_test_y+y_subtraction, "ro", label="prediction")
plt.legend()
plt.show()

## Modify get_column_combinations to also allow for specifying target column(s)

In [ ]:
def get_column_combinations_w_targets(all_cols, inputs, outputs, targets):
    
    assert inputs+outputs <= len(all_cols), "More input and output columns specified than there are columns."
    
    # initialise final list of column combinations
    col_combinations = []
    
    if targets:
        all_input_cols = [x for x in all_cols if x not in targets]
    else:
        all_input_cols = all_cols
        
    # first, draw possible input tuples
    input_combinations = list(itertools.combinations(all_input_cols, inputs))
    # now go through all possible input combinations
    for i in input_combinations:
        if targets:
            curr_output_cols = targets
        else:
            # get list of possible output columns, i.e. columns that are not yet part of current input columns
            curr_output_cols = [o for o in all_cols if o not in i]
        # now draw from that list all currently possible output combinations
        output_combinations = list(itertools.combinations(curr_output_cols, outputs))
        # add all currently possible output combinations to the current input columns and save in final list
        for oc in output_combinations:
            col_combinations.append(i+(*oc,))
        
    return col_combinations

In [ ]:
get_column_combinations(["a", "b", "c", "d", "e"], 3, 2)

In [ ]:
get_column_combinations_w_targets(["a", "b", "c", "d", "e"], 2, 2, ["a", "b"])

# The main routine

In [ ]:
def predictability(data, input_cols=1, output_cols=1, col_set=None, primkey_cols=[], targets=[],
                   method="MLP", hidden_layers=None, alphas=None, scoring="r2", scaling=True, 
                   max_iter=10000, n_jobs=-1, verbose=1):
    
    # TODO: map scoring to possible options
    scoring_dict = {
        "r2": "r2",
        "MAPE": "neg_mean_absolute_percentage_error",
        "neg_mean_absolute_percentage_error": "neg_mean_absolute_percentage_error",
        "RMSE": "neg_root_mean_squared_error",
        "neg_root_mean_squared_error": "neg_root_mean_squared_error",
        "MAE": "neg_mean_absolute_error",
        "neg_mean_absolute_error": "neg_mean_absolute_error"
    }
    scoring = scoring_dict[scoring]
    
    # if we want to measure the overall time
    start = time.time()
    
    # initialise the dictionary that is going to save the metrics per tuple
    metric_dict = {}
    
    # dict to save x-/y-train/-test and predicted values for subsequent plotting
    data_dict = {}
    
    # if primary keys are fed in, data columns should not contain these
    data_cols = [col for col in data.columns.to_list() if col not in primkey_cols]
    
    # if set of columns that should be considered is fed in, use this
    if col_set is not None:
        data_cols = list(set(col_set))
    
    # get the list of tuples of input and output columns
    if targets:
        data_tuples = get_column_combinations_w_targets(data_cols, input_cols, output_cols, targets)
    else:
        data_tuples = get_column_combinations(data_cols, input_cols, output_cols)    
    
    # for printing the progress of the analysis
    counter_tuples = 0
    
    # go through all tuples
    # or testing subset only:
    #data_tuples = [("Electric power consumption (kWh per capita)", "Life expectancy at birth, total (years)")]+data_tuples[:5]
    #
    for curr_tuple in data_tuples:
        
        # if we want to measure the current tuple's analysis time
        curr_start = time.time()
        
        print("Analysing "+str(curr_tuple)+" now.")
        
        # TODO: implement going through all permutations
        
        # get current inputs and outputs
        curr_inputs = list(curr_tuple[:input_cols])
        curr_outputs = list(curr_tuple[input_cols:])
        
        # reduce data to current columns and drop NAs
        curr_data = data[curr_inputs+curr_outputs].dropna()
        
        # dict of min values of current columns, used for power law fitting
        mi_dict = {}
        for col in curr_inputs+curr_outputs:
            curr_min = min(curr_data[col])
            if curr_min < 0:
                x_subtraction = 1.05*curr_min
            elif curr_min == 0:
                x_subtraction = -0.05
            else:
                x_subtraction = 0
            mi_dict[col] = x_subtraction
            
        
        # do data preparations and train-test-split
        curr_X_train, curr_X_test, curr_y_train, curr_y_test = data_prep_split(curr_data, curr_inputs, curr_outputs)
        
        # compute standard deviation of curr_y_test for later scaling of the RMSE
        curr_y_test_std = np.std(curr_y_test)
        
        #
        # y-mean "prediction"
        #
        curr_y_train_mean = np.mean(curr_y_train)
        curr_y_test_pred_mean = curr_y_train_mean*np.ones(len(curr_X_test))
        # metrics
        curr_mean_r2 = r2_score(curr_y_test, curr_y_test_pred_mean)
        curr_mean_rmse = mean_squared_error(curr_y_test, curr_y_test_pred_mean, squared=False)
        curr_mean_mape = mean_absolute_percentage_error(curr_y_test, curr_y_test_pred_mean)
        curr_mean_rae = rae(curr_y_test, curr_y_test_pred_mean)
        curr_mean_dcor = dcor.distance_correlation(curr_y_test, curr_y_test_pred_mean)
        
        #
        # linear regression
        #
        lin_reg = LinearRegression().fit(curr_X_train,curr_y_train)
        curr_y_test_pred = lin_reg.predict(curr_X_test)
        # metrics
        curr_lin_r2 = r2_score(curr_y_test, curr_y_test_pred)
        curr_lin_rmse = mean_squared_error(curr_y_test, curr_y_test_pred, squared=False)
        curr_lin_mape = mean_absolute_percentage_error(curr_y_test, curr_y_test_pred)
        curr_lin_rae = rae(curr_y_test, curr_y_test_pred)
        curr_lin_dcor = dcor.distance_correlation(curr_y_test, curr_y_test_pred)
        
        '''
        # power law fit so far only for 1-1 connections implemented
        if input_cols==1 and output_cols==1:
            #
            # power law fit
            #
            curr_y_test_pred_pl = fit_predict_power_law_1_1(curr_X_train, curr_X_test, curr_y_train, curr_y_test)

            # metrics
            curr_pl_r2 = r2_score(curr_y_test, curr_y_test_pred_pl)
            curr_pl_rmse = mean_squared_error(curr_y_test, curr_y_test_pred_pl, squared=False)
            curr_pl_mape = mean_absolute_percentage_error(curr_y_test, curr_y_test_pred_pl)
            curr_pl_rae = rae(curr_y_test, curr_y_test_pred_pl)
            curr_pl_dcor = dcor.distance_correlation(curr_y_test, curr_y_test_pred_pl)
            '''
        #
        # power law fit
        #
        if ((curr_X_train>0).all().all()) and ((curr_X_test>0).all().all()):
            do_pl_fit = True
        else:
            do_pl_fit = False
        
        if do_pl_fit:
            
            curr_X_train_log = curr_X_train.apply(lambda x: np.log(x))
            curr_X_test_log = curr_X_test.apply(lambda x: np.log(x))
            curr_y_train_log = curr_y_train.apply(lambda x: np.log(x))
            #curr_y_test_log = curr_y_test.apply(lambda x: np.log(x))
            
            pl_fit = LinearRegression().fit(curr_X_train_log, curr_y_train_log)
            
            curr_y_test_pred_pl = pl_fit.predict(curr_X_test_log)
            
            curr_y_test_pred_pl = np.exp(curr_y_test_pred_pl)
            
            # metrics
            curr_pl_r2 = r2_score(curr_y_test, curr_y_test_pred_pl)
            curr_pl_rmse = mean_squared_error(curr_y_test, curr_y_test_pred_pl, squared=False)
            curr_pl_mape = mean_absolute_percentage_error(curr_y_test, curr_y_test_pred_pl)
            curr_pl_rae = rae(curr_y_test, curr_y_test_pred_pl)
            curr_pl_dcor = dcor.distance_correlation(curr_y_test, curr_y_test_pred_pl)
            
        
        #
        # MLP regression
        print("start MLP routine")
        #
        # list of hidden layer sizes for GridSearch
        if hidden_layers is None:
            hidden_layers = [(12,), 
                              (50,), 
                              (70,5,), 
                              #(40,18,3,)
                            ]
        # list of alpha values for GridSearch
        if alphas is None:
            alphas = [0.001, 
                      0.0001, 
                      0.00001
                     ]

        # via pipeline (with and without scaler)
        if scaling=="yes":
            pipe = Pipeline([
                            ('scaler', StandardScaler()),
                            ('mlp', MLPRegressor(max_iter=max_iter))
                            ])
            pipe_params = [
                           {'mlp__hidden_layer_sizes': hidden_layers,
                            'mlp__alpha': alphas}
                            ]
            clf = GridSearchCV(pipe,
                               param_grid=pipe_params,
                               cv=3,
                               scoring=scoring,
                               return_train_score=True,
                               verbose=verbose,
                               n_jobs=n_jobs
                              )        
        elif scaling=="no":
            pipe = Pipeline([
                            ('scaler', StandardScaler()),
                            ('mlp', MLPRegressor(max_iter=max_iter))
                            ])
            pipe_params = [{'scaler': ['passthrough'],
                            'mlp__hidden_layer_sizes': hidden_layers,
                            'mlp__alpha': alphas}]
            clf = GridSearchCV(pipe,
                               param_grid=pipe_params,
                               cv=3,
                               scoring=scoring,
                               return_train_score=True,
                               verbose=verbose,
                               n_jobs=n_jobs
                              )
        else:
            pipe = Pipeline([
                            ('scaler', StandardScaler()),
                            ('mlp', MLPRegressor(max_iter=max_iter))
                            ])
            pipe_params = [{'scaler': ['passthrough'],
                            'mlp__hidden_layer_sizes': hidden_layers,
                            'mlp__alpha': alphas}, 
                           {'mlp__hidden_layer_sizes': hidden_layers,
                            'mlp__alpha': alphas}]
            clf = GridSearchCV(pipe,
                               param_grid=pipe_params,
                               cv=3,
                               scoring=scoring,
                               return_train_score=True,
                               verbose=verbose,
                               n_jobs=n_jobs
                              )
        
        clf.fit(curr_X_train, curr_y_train)
        
        curr_best_params = clf.best_params_
        curr_y_test_pred = clf.predict(curr_X_test)
        
        # metrics
        curr_mlp_r2 = r2_score(curr_y_test, curr_y_test_pred)
        curr_mlp_rmse = mean_squared_error(curr_y_test, curr_y_test_pred, squared=False)
        curr_mlp_mape = mean_absolute_percentage_error(curr_y_test, curr_y_test_pred)
        curr_mlp_rae = rae(curr_y_test, curr_y_test_pred)
        curr_mlp_dcor = dcor.distance_correlation(curr_y_test, curr_y_test_pred)

        # save metrics into dict
        # power law fit so far only for 1-1 connections implemented
        if do_pl_fit:
            metric_dict[curr_tuple] = {"MLP r2": curr_mlp_r2, "linear r2": curr_lin_r2, 
                                       "pow. law r2": curr_pl_r2, "mean r2": curr_mean_r2, 
                                        "MLP RMSE": curr_mlp_rmse, "linear RMSE": curr_lin_rmse,
                                        "pow. law RMSE": curr_pl_rmse, "mean RMSE": curr_mean_rmse,
                                        "MLP RMSE/std": curr_mlp_rmse/curr_y_test_std, "linear RMSE/std": curr_lin_rmse/curr_y_test_std,
                                       "pow. law RMSE/std": curr_pl_rmse/curr_y_test_std, "mean RMSE/std": curr_mean_rmse/curr_y_test_std,
                                        "MLP MAPE": curr_mlp_mape, "linear MAPE": curr_lin_mape,
                                       "pow. law MAPE": curr_pl_mape, "mean MAPE": curr_mean_mape,
                                        "MLP rae": curr_mlp_rae, "linear rae": curr_lin_rae,
                                        "pow. law rae": curr_pl_rae, "mean rae": curr_mean_rae,
                                        "MLP dcor": curr_mlp_dcor, "linear dcor": curr_lin_dcor,
                                        "pow. law dcor": curr_pl_dcor, "mean dcor": curr_mean_dcor,
                                               }
        else:
            metric_dict[curr_tuple] = {"MLP r2": curr_mlp_r2, "linear r2": curr_lin_r2, "mean r2": curr_mean_r2, 
                                        "MLP RMSE": curr_mlp_rmse, "linear RMSE": curr_lin_rmse, "mean RMSE": curr_mean_rmse,
                                        "MLP RMSE/std": curr_mlp_rmse/curr_y_test_std, "linear RMSE/std": curr_lin_rmse/curr_y_test_std,
                                        "mean RMSE/std": curr_mean_rmse/curr_y_test_std,
                                        "MLP MAPE": curr_mlp_mape, "linear MAPE": curr_lin_mape, "mean MAPE": curr_mean_mape,
                                        "MLP rae": curr_mlp_rae, "linear rae": curr_lin_rae, "mean rae": curr_mean_rae,
                                        "MLP dcor": curr_mlp_dcor, "linear dcor": curr_lin_dcor, "mean dcor": curr_mean_dcor,
                                               }

        # save values into dict
        # power law fit so far only for 1-1 connections implemented
        if do_pl_fit:
            
            data_dict[curr_tuple] = {"X_train": curr_X_train, "X_test": curr_X_test,
                                     "y_train": curr_y_train, "y_test": curr_y_test, "y_test_pred": curr_y_test_pred,
                                     "y_test_pred_pl": curr_y_test_pred_pl, "y_test_pred_mean": curr_y_test_pred_mean,
                                     "GridSearchParams": curr_best_params, "scores": clf.cv_results_
                                    }
        else:
            
            data_dict[curr_tuple] = {"X_train": curr_X_train, "X_test": curr_X_test,
                                     "y_train": curr_y_train, "y_test": curr_y_test, "y_test_pred": curr_y_test_pred,
                                     "y_test_pred_mean": curr_y_test_pred_mean,
                                     "GridSearchParams": curr_best_params, "scores": clf.cv_results_
                                    }
        
                
        # for printing the CV results per tuple
        #print(clf.cv_results_)
        
        print("The analysis of this tuple took "+str(round(time.time()-curr_start,2))+"s.")
        # for printing the progress of the analysis
        counter_tuples += 1
        print("-----"+str(counter_tuples)+"/"+str(len(data_tuples))+"-----")
    
    print("The whole run took "+str(round(time.time()-start,2))+"s.")
    
    return metric_dict, data_dict

In [ ]:
metrics, datas = predictability(data=df2007,
                                input_cols=1,
                                output_cols=1,
                                col_set = ['Electric power consumption (kWh per capita)', 
                                           'Agriculture, value added (% of GDP)',
                                           'Life expectancy at birth, total (years)', 
                                           'CO2 emissions (metric tons per capita)'],
                                primkey_cols = prim_keys,
                                scoring="RMSE"
                               )

In [ ]:
metrics_df = pd.DataFrame.from_dict(metrics).transpose()

# or 3d ones
#metrics_df = pd.DataFrame.from_dict(metrics3d).transpose()

metrics_df

In [ ]:
# 3-1 connections metrics
metrics3_1_df = pd.DataFrame.from_dict(metrics3_1).transpose()
# may show MLP metrics only
metrics3_1_df[["MLP r2", "MLP RMSE", "MLP MAPE", "MLP rae", "MLP dcor"]].sort_values(by="MLP r2", ascending=False)

### Run the plotting routine alone

issues:
* [-2]: r2 of power law fit = 0 ! But dcor is good. MAPE high due to one y-value = 0.
* [5]: r2 of MLP = 0 ! But actually, w.r.t. training data a rather good fit. dcor is good, too!
* [-3]: bad MLP fit, but MAPE rather good due to y-values far away from 0.
* [-4]: similar to above, MAPE again okay as high deviations exist mainly for predictions of high y-values

In [ ]:
plot_2d_result(list(datas.keys())[-2], metrics, datas, show=True)

In [ ]:
#plot_3d_result(list(datas.keys())[2], metrics, datas, show=True)
plot_3d_result(list(datas3d.keys())[11], metrics3d, datas3d, show=True)

### save dicts

In [ ]:
# 3d data
with open('metrics3d.pkl', 'wb') as f:
    pickle.dump(metrics, f)
with open('datas3d.pkl', 'wb') as f:
    pickle.dump(datas, f)
#with open('plots3d.pkl', 'wb') as f:
#    pickle.dump(plots, f)

In [ ]:
# 2d data
with open('metrics.pkl', 'wb') as f:
    pickle.dump(metrics, f)
with open('datas.pkl', 'wb') as f:
    pickle.dump(datas, f)
#with open('plots.pkl', 'wb') as f:
#    pickle.dump(plots, f)

### load dicts

In [ ]:
# 2d data
with open('metrics.pkl', 'rb') as f:
    metrics = pickle.load(f)
with open('datas.pkl', 'rb') as f:
    datas = pickle.load(f)
#with open('plots.pkl', 'rb') as f:
#    plots = pickle.load(f)


In [ ]:
# 3d data
with open('metrics3d.pkl', 'rb') as f:
    metrics3d = pickle.load(f)
with open('datas3d.pkl', 'rb') as f:
    datas3d = pickle.load(f)
#with open('plots3d.pkl', 'rb') as f:
#    plots = pickle.load(f)


In [ ]:
# data
with open('metrics3_1.pkl', 'rb') as f:
    metrics3_1 = pickle.load(f)
with open('datas3_1.pkl', 'rb') as f:
    datas3_1 = pickle.load(f)
#with open('plots3d.pkl', 'rb') as f:
#    plots = pickle.load(f)


#### or backups

In [ ]:


with open('metrics_backup.pkl', 'rb') as f:
    metrics_backup = pickle.load(f)
with open('datas_backup.pkl', 'rb') as f:
    datas_backup = pickle.load(f)
with open('plots_backup.pkl', 'rb') as f:
    plots_backup = pickle.load(f)


### Big run on 4-1 connections

In [ ]:
metrics_4_1, datas_4_1 = predictability(data=df2007,
                                input_cols=4,
                                output_cols=1,
                                primkey_cols = prim_keys,
                                scoring="RMSE"
                               )

In [ ]:
datas_4_1[('Agriculture, value added (% of GDP)',
  'CO2 emissions (metric tons per capita)',
  'Domestic credit provided by financial sector (% of GDP)',
  'Electric power consumption (kWh per capita)',
  'Exports of goods and services (% of GDP)')]

In [ ]:
with open('metrics_4_1.pkl', 'wb') as f:
    pickle.dump(metrics_4_1, f)
with open('datas_4_1.pkl', 'wb') as f:
    pickle.dump(datas_4_1, f)

## Big run to check dependency on scoring choice during GridSearchCV

In [ ]:
# big run through all 4 scorings
metrics_scorings = {}
for scoring in ["RMSE", "MAPE", "r2", "MAE"]:
    curr_metrics, curr_datas = predictability(data=df2007,
                                input_cols=1,
                                output_cols=1,
                                primkey_cols = prim_keys,
                                scoring=scoring
                               )
    metrics_scorings[scoring] = curr_metrics

In [ ]:
with open('metrics_scorings.pkl', 'wb') as f:
    pickle.dump(metrics_scorings, f)

In [ ]:
metrics_scorings_df = pd.DataFrame.from_dict(metrics_scorings)#.transpose()
RMSE_metrics = metrics_scorings_df['RMSE'].apply(pd.Series)[["MLP r2", "MLP RMSE", "MLP MAPE", "MLP rae", "MLP dcor"]]
RMSE_metrics.columns = pd.MultiIndex.from_product([["RMSE scoring"], RMSE_metrics.columns])
all_metrics_df = RMSE_metrics.copy(deep=True)
for scoring in ["r2", "MAPE", "MAE"]:
    curr_metrics_df = metrics_scorings_df[scoring].apply(pd.Series)[["MLP r2", "MLP RMSE", "MLP MAPE", "MLP rae", "MLP dcor"]]
    curr_metrics_df.columns = pd.MultiIndex.from_product([[scoring+" scoring"], curr_metrics_df.columns])
    all_metrics_df = pd.concat([all_metrics_df, curr_metrics_df], axis=1)
all_metrics_df

## Tests and checks

### all combinatinos for, e.g., 2-1 connections

In [ ]:
all_cols = ["a", "b", "c", 
            "d",# "e", "f"
           ]
inputs = 2
outputs = 1
def get_column_combinations(all_cols, inputs, outputs):
    
    assert inputs+outputs <= len(all_cols), "More input and output columns specified than there are columns."
    
    # initialise final list of column combinations
    col_combinations = []
    # first, draw possible input tuples
    input_combinations = list(itertools.combinations(all_cols, inputs))
    # now go through all possible input combinations
    for i in input_combinations:
        # get list of possible output columns, i.e. columns that are not yet part of current input columns
        curr_output_cols = [o for o in all_cols if o not in i]
        # now draw from that list all currently possible output combinations
        output_combinations = list(itertools.combinations(curr_output_cols, outputs))
        # add all currently possible output combinations to the current input columns and save in final list
        for oc in output_combinations:
            col_combinations.append(i+(*oc,))
        
    return col_combinations

In [ ]:
get_column_combinations(all_cols, 2, 1)

### other tests

In [ ]:
test_df = df2007[["Electric power consumption (kWh per capita)", "Life expectancy at birth, total (years)"]].dropna()
test_x = test_df[["Electric power consumption (kWh per capita)"]].values.flatten()
test_y = test_df[["Life expectancy at birth, total (years)"]].values.flatten()


In [ ]:
cols = ['Agriculture, value added (% of GDP)', 'Domestic credit provided by financial sector (% of GDP)']
test_df = df2007[cols].dropna()
x_subtraction = 1.05*min(test_df[cols[0]].values)-0.01
y_subtraction = 1.05*min(test_df[cols[1]].values)-0.01
test_x = test_df[cols[0]].values.flatten()-x_subtraction
test_y = test_df[cols[1]].values.flatten()-y_subtraction
print(x_subtraction, y_subtraction)

In [ ]:
test_params = fit_power_law_1_1(test_x, test_y)

In [ ]:
test_params

In [ ]:
pred_test_y = power_law_1_1(test_x, *test_params)

In [ ]:
plt.plot(test_x+x_subtraction, test_y+y_subtraction, "bo", label="data")
plt.plot(test_x+x_subtraction, pred_test_y+y_subtraction, "ro", label="prediction")
plt.legend()
plt.show()

## Modify get_column_combinations to also allow for specifying target column(s)

In [ ]:
def get_column_combinations_w_targets(all_cols, inputs, outputs, targets):
    
    assert inputs+outputs <= len(all_cols), "More input and output columns specified than there are columns."
    
    # initialise final list of column combinations
    col_combinations = []
    
    if targets:
        all_input_cols = [x for x in all_cols if x not in targets]
    else:
        all_input_cols = all_cols
        
    # first, draw possible input tuples
    input_combinations = list(itertools.combinations(all_input_cols, inputs))
    # now go through all possible input combinations
    for i in input_combinations:
        if targets:
            curr_output_cols = targets
        else:
            # get list of possible output columns, i.e. columns that are not yet part of current input columns
            curr_output_cols = [o for o in all_cols if o not in i]
        # now draw from that list all currently possible output combinations
        output_combinations = list(itertools.combinations(curr_output_cols, outputs))
        # add all currently possible output combinations to the current input columns and save in final list
        for oc in output_combinations:
            col_combinations.append(i+(*oc,))
        
    return col_combinations

In [ ]:
get_column_combinations(["a", "b", "c", "d", "e"], 3, 2)

In [ ]:
get_column_combinations_w_targets(["a", "b", "c", "d", "e"], 2, 2, ["a", "b"])